In [20]:
TOK_JSON = """
{"\u0251": 1, "\u00e6": 2, "\u028c": 3, "a\u028a": 4, "\u0259": 5, "\u025d": 6, "a\u026a": 7, "b": 8, "\u02a7": 9, "d": 10, "\u00f0": 11, "\u027e": 12, "\u025b": 13, "l\u0329": 14, "m\u0329": 15, "n\u0329": 16, "\u014b\u0329": 17, "\u025a": 18, "e\u026a": 19, "f": 20, "g": 21, " ": 22, "h": 24, "\u026a": 26, "i\u02d0": 27, "\u02a4": 28, "k": 29, "l": 30, "m": 31, "n": 32, "\u014b": 33, "\u027e\u0303": 34, "o\u028a": 35, "\u0254\u026a": 36, "p": 37, "ʔ": 38, "\u0279": 39, "s": 40, "\u0283": 41, "t": 42, "\u03b8": 43, "\u028a": 44, "u\u02d0": 45, "v": 46, "w": 47, "j": 48, "z": 49, "\u0292": 50, "|": 0, "[UNK]": 51, "[PAD]": 52}
"""

In [21]:
import json

TOKENS = json.loads(TOK_JSON)
TOKLIST = ["\[UNK\]"]
for key in TOKENS:
    if "[" in key or key == "|" or key == " ":
        continue
    TOKLIST.append(key)
# sort TOKLIST by length, longest first
TOKLIST.sort(key=len, reverse=True)
TOK_REGEX_INNER = "|".join(TOKLIST)
TOK_REGEX = fr"({TOK_REGEX_INNER})"

In [25]:
import re

def tokenise(text):
    tokens = []
    text = text.strip()
    while text:
        match = re.match(TOK_REGEX, text)
        if not match:
            raise ValueError(f"Could not match token in text: {text}")
        token = match.group(0)
        if token == "[UNK]":
            tokens.append("ɪ")
        else:
            tokens.append(token)
        text = text[len(token):].strip()
    return tokens

In [23]:
SAMPLE_TIMIT = "doʊn t biːɪf ɹeɪʔ l[UNK]ɾl̩wʌn ð[UNK]biːs kæn t[UNK]ɚʔʧuːwɑlʔaɪmɚɹaʊn dɪnðɛnhiːt[UNK]tɪz kæp ʔoʊvɝhɪz lɛf tʔiːɝ ʔɪnʃʊk[UNK]z k lʌb ʔɪʔðɪp ɹ[UNK]n ʔ s"

In [26]:
tokenise(SAMPLE_TIMIT)

['d',
 'oʊ',
 'n',
 't',
 'b',
 'iː',
 'ɪ',
 'f',
 'ɹ',
 'eɪ',
 'ʔ',
 'l',
 'ɪ',
 'ɾ',
 'l̩',
 'w',
 'ʌ',
 'n',
 'ð',
 'ɪ',
 'b',
 'iː',
 's',
 'k',
 'æ',
 'n',
 't',
 'ɪ',
 'ɚ',
 'ʔ',
 'ʧ',
 'uː',
 'w',
 'ɑ',
 'l',
 'ʔ',
 'aɪ',
 'm',
 'ɚ',
 'ɹ',
 'aʊ',
 'n',
 'd',
 'ɪ',
 'n',
 'ð',
 'ɛ',
 'n',
 'h',
 'iː',
 't',
 'ɪ',
 't',
 'ɪ',
 'z',
 'k',
 'æ',
 'p',
 'ʔ',
 'oʊ',
 'v',
 'ɝ',
 'h',
 'ɪ',
 'z',
 'l',
 'ɛ',
 'f',
 't',
 'ʔ',
 'iː',
 'ɝ',
 'ʔ',
 'ɪ',
 'n',
 'ʃ',
 'ʊ',
 'k',
 'ɪ',
 'z',
 'k',
 'l',
 'ʌ',
 'b',
 'ʔ',
 'ɪ',
 'ʔ',
 'ð',
 'ɪ',
 'p',
 'ɹ',
 'ɪ',
 'n',
 'ʔ',
 's']